<a href="https://colab.research.google.com/github/Rishav-hub/Auto-Gluon/blob/main/04_AutoGluon_for_Image_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation
AutoGluon requires Python version 3.7, 3.8, or 3.9 [link text](https://auto.gluon.ai/stable/index.html#installation)


In [ ]:
!pip3 install autogluon

In [1]:
!pip3 install "torch>=1.0,<1.11+cu113" -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [2]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0
print(torch.__version__)

True
1
1.10.2+cu102


## Properly load any image dataset as ImageDataset

There are generally three ways to load a dataset for ImagePredictor:

- Load a csv file or construct your own pandas DataFrame 
with image and label columns

- Load a image folder directly with ImageDataset

- Convert a list of images into a dataset directly with ImageDataset

### Load a csv file or construct a DataFrame object

You may use any tabular data as long as you can create image(absolute or relative paths to images) and label(category for each image) columns.

In [3]:
%matplotlib inline
import autogluon.core as ag
from autogluon.vision import ImageDataset
import pandas as pd

In [4]:
csv_file = ag.utils.download('https://autogluon.s3-us-west-2.amazonaws.com/datasets/petfinder_example.csv')
df = pd.read_csv(csv_file)
df.head()

100%|██████████| 820/820 [00:00<00:00, 1974.05KB/s]


,image,PetID,label
0,petfinder_data/train_images/015da9e87-1.jpg,015da9e87,0
1,petfinder_data/train_images/022606901-1.jpg,022606901,0
2,petfinder_data/train_images/02f89bdcb-1.jpg,02f89bdcb,0
3,petfinder_data/train_images/03f217352-1.jpg,03f217352,0
4,petfinder_data/train_images/040a9a6f9-1.jpg,040a9a6f9,0


If the image paths are not relative to the current working directory, you may use the helper function to prepend a prefix for each image.

In [5]:
df = ImageDataset.from_csv(csv_file, root='/content')
df.head()

,image,PetID,label
0,/content/petfinder_data/train_images/015da9e87...,015da9e87,0
1,/content/petfinder_data/train_images/022606901...,022606901,0
2,/content/petfinder_data/train_images/02f89bdcb...,02f89bdcb,0
3,/content/petfinder_data/train_images/03f217352...,03f217352,0
4,/content/petfinder_data/train_images/040a9a6f9...,040a9a6f9,0


### Load an image directory

It’s pretty common that sometimes you only have a folder of images, organized by the category names. Recursively looping through images is tedious. You can use ImageDataset.from_folders or ImageDataset.from_folder to avoid implementing recursive search.

In [6]:
train_data, _, test_data = ImageDataset.from_folders('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip', train='train', test='test')
print('train #', len(train_data), 'test #', len(test_data))
train_data.head()

100%|██████████| 40895/40895 [00:02<00:00, 19672.32KB/s]


data/
├── test/
└── train/
train # 800 test # 80


,image,label
0,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
1,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
2,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
3,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
4,/root/.gluoncv/datasets/shopee-iet/data/train/...,0


### If you have a folder without train or test root folders


In [7]:
# use the train from shopee-iet as new root
import os
root = os.path.join(os.path.dirname(train_data.iloc[0]['image']), '..')
all_data = ImageDataset.from_folder(root)
all_data.head()

,image,label
0,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
1,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
2,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
3,/root/.gluoncv/datasets/shopee-iet/data/train/...,0
4,/root/.gluoncv/datasets/shopee-iet/data/train/...,0


### Convert a list of images to dataset

In [8]:
# We use the Oxford-IIIT Pet Dataset mini pack as an example
pets = ag.utils.download('https://autogluon.s3-us-west-2.amazonaws.com/datasets/oxford-iiit-pet-mini.zip')
pets = ag.utils.unzip(pets)
image_list = [x for x in os.listdir(os.path.join(pets, 'images')) if x.endswith('jpg')]
def label_fn(x):
    return 'cat' if os.path.basename(x)[0].isupper() else 'dog'
new_data = ImageDataset.from_name_func(image_list, label_fn, root=os.path.join(os.getcwd(), pets, 'images'))
new_data

100%|██████████| 35730/35730 [00:01<00:00, 20403.77KB/s]


,image,label
0,/content/oxford-iiit-pet/images/american_pit_b...,0
1,/content/oxford-iiit-pet/images/staffordshire_...,0
2,/content/oxford-iiit-pet/images/boxer_177.jpg,0
3,/content/oxford-iiit-pet/images/saint_bernard_...,0
4,/content/oxford-iiit-pet/images/japanese_chin_...,0
...,...,...
95,/content/oxford-iiit-pet/images/basset_hound_4...,0
96,/content/oxford-iiit-pet/images/Birman_22.jpg,1
97,/content/oxford-iiit-pet/images/Maine_Coon_262...,1
98,/content/oxford-iiit-pet/images/english_cocker...,0


## Visualize images

In [ ]:
new_data.show_images()


## Image Prediction

In [10]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset

### Create Image Dataset

In [35]:
ls

In [36]:
train_dataset, _, test_dataset = ImageDataset.from_folders('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip')
print(train_dataset)

data/
├── test/
└── train/
                                                 image  label
0    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
1    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
2    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
3    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
4    /root/.gluoncv/datasets/shopee-iet/data/train/...      0
..                                                 ...    ...
795  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
796  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
797  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
798  /root/.gluoncv/datasets/shopee-iet/data/train/...      3
799  /root/.gluoncv/datasets/shopee-iet/data/train/...      3

[800 rows x 2 columns]


### Use AutoGluon to Fit Models

In [37]:
predictor = ImagePredictor()
# since the original dataset does not provide validation split, the `fit` function splits it randomly with 90/10 ratio
predictor.fit(train_dataset, hyperparameters={'epochs': 2})  # you can trust the default config, we reduce the # epoch to save some build time

ImagePredictor sets accuracy as default eval_metric for classification problems.
`time_limit=auto` set to `time_limit=7200`.
Reset labels to [0, 1, 2, 3]
Randomly split train_data into train[720]/validation[80] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.misc.num_workers 4 != 2
root.misc.seed       42 != 421
root.img_cls.model   resnet101 != resnet50
root.train.epochs    200 != 2
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
}
Saved config to /root/bffa535a/.trial_0/config.yaml
Model resnet50 created, param count:                                         23516228
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
[Epoch 0] training: accuracy=0.255556
[Epoch 0] speed: 86 samples/sec	time cost: 

In [38]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.389, val acc: 0.662


### Predict on a New Image

In [39]:
image_path = test_dataset.iloc[0]['image']
result = predictor.predict(image_path)
print(result)

0    1
Name: label, dtype: int64


In [40]:
# Probability
proba = predictor.predict_proba(image_path)
print(proba)

          0         1         2         3
0  0.235921  0.300823  0.255399  0.207857


In [ ]:
# You can also feed in multiple images all together
bulk_result = predictor.predict(test_dataset)
print(bulk_result)

### Generate image features with a classifier

In [42]:
image_path = test_dataset.iloc[0]['image']
feature = predictor.predict_feature(image_path)
print(feature)

                                       image_feature  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                               image  
0  /root/.gluoncv/datasets/shopee-iet/data/test/B...  


### Evaluate on Test Dataset

In [50]:
test_acc = predictor.evaluate(test_dataset)
print('Top-1 test acc: %.3f' % test_acc['top1'])

[Epoch 1] validation: top1=0.637500 top5=1.000000


Top-1 test acc: 0.637


### Save and load classifiers

In [52]:
filename = '/content/predictor.ag'
predictor.save(filename)
predictor_loaded = ImagePredictor.load(filename)
# use predictor_loaded as usual
result = predictor_loaded.predict(image_path)
print(result)

0    1
Name: label, dtype: int64
